In [1]:
import tensorflow as tf

import numpy as np
import os
import time

In [2]:
path_to_file = 'trump_18June2020.txt'

In [3]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print(f'Length of text: {len(text)} characters')


Length of text: 176072 characters


In [4]:
example_texts = ['abcdefg', 'xyz']

chars = tf.strings.unicode_split(example_texts, input_encoding='UTF-8')
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [5]:
ids_from_chars = tf.keras.layers.StringLookup(
    vocabulary=list(vocab), mask_token=None)

NameError: name 'vocab' is not defined

In [ ]:
ids = ids_from_chars(chars)
ids

<tf.RaggedTensor [[54, 55, 56, 57, 58, 59, 60], [77, 78, 79]]>

In [ ]:
tf.strings.reduce_join(chars, axis=-1).numpy()

array([b'abcdefg', b'xyz'], dtype=object)

In [ ]:
def text_from_ids(ids):
  return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

In [ ]:
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
all_ids

<tf.Tensor: shape=(176072,), dtype=int64, numpy=array([ 1, 40, 61, ..., 58, 53,  1], dtype=int64)>

In [ ]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

In [ ]:
for ids in ids_dataset.take(10):
    print(chars_from_ids(ids).numpy().decode('utf-8'))



O
h
,
 
w
e
 
h
a


In [ ]:
seq_length = 100

In [ ]:
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

for seq in sequences.take(1):
  print(chars_from_ids(seq))

tf.Tensor(
[b'\n' b'O' b'h' b',' b' ' b'w' b'e' b' ' b'h' b'a' b'd' b' ' b's' b'u'
 b'c' b'h' b' ' b'l' b'u' b'c' b'k' b' ' b'i' b'n' b' ' b'O' b'r' b'l'
 b'a' b'n' b'd' b'o' b',' b' ' b'w' b'e' b' ' b'l' b'o' b'v' b'e' b' '
 b'b' b'e' b'i' b'n' b'g' b' ' b'i' b'n' b' ' b'O' b'r' b'l' b'a' b'n'
 b'd' b'o' b'.' b' ' b'T' b'h' b'a' b'n' b'k' b' ' b'y' b'o' b'u' b'.'
 b' ' b'T' b'h' b'a' b'n' b'k' b' ' b'y' b'o' b'u' b',' b' ' b'O' b'r'
 b'l' b'a' b'n' b'd' b'o' b'.' b' ' b'W' b'h' b'a' b't' b' ' b'a' b' '
 b't' b'u' b'r'], shape=(101,), dtype=string)


In [ ]:
for seq in sequences.take(5):
  print(text_from_ids(seq).numpy())

b'\nOh, we had such luck in Orlando, we love being in Orlando. Thank you. Thank you, Orlando. What a tur'
b'nout! What a turnout! You know this is a very big arena for a Tuesday night. I said, you know, if we '
b'have about three or four empty seats, the fake news will say, Headlines: "He didn\'t fill up the arena'
b'", so I said, maybe we shouldn\'t take a chance.\nMaybe we shouldn\'t go to Orlando as you go someplace '
b'else, then I said, "No, I think, we\'ll go to Orlando." And -- and not only did we fill it up, but we '


In [ ]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [ ]:
split_input_target(list("Tensorflow"))

(['T', 'e', 'n', 's', 'o', 'r', 'f', 'l', 'o'],
 ['e', 'n', 's', 'o', 'r', 'f', 'l', 'o', 'w'])

In [ ]:
dataset = sequences.map(split_input_target)

In [ ]:
for input_example, target_example in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

Input : b'\nOh, we had such luck in Orlando, we love being in Orlando. Thank you. Thank you, Orlando. What a tu'
Target: b'Oh, we had such luck in Orlando, we love being in Orlando. Thank you. Thank you, Orlando. What a tur'


In [ ]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<PrefetchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

In [ ]:
# Length of the vocabulary in StringLookup Layer
vocab_size = len(ids_from_chars.get_vocabulary())

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [ ]:
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

In [ ]:
model = MyModel(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

In [ ]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 84) # (batch_size, sequence_length, vocab_size)


In [ ]:
model.summary()

Model: "my_model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     multiple                  21504     
                                                                 
 gru_1 (GRU)                 multiple                  3938304   
                                                                 
 dense_4 (Dense)             multiple                  86100     
                                                                 
Total params: 4,045,908
Trainable params: 4,045,908
Non-trainable params: 0
_________________________________________________________________


In [ ]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

In [ ]:
sampled_indices

array([40, 61, 17, 74, 31, 69, 54, 44, 13, 72, 21, 27, 37, 79, 25, 78, 52,
       21,  5, 10, 41, 68, 26, 48, 53, 15, 22, 22, 70, 63, 46, 29, 44, 26,
       81, 69, 40, 32, 14, 80, 14, 81, 80, 63, 16, 46,  2,  8, 25,  6,  1,
        1, 66, 41, 71, 71, 80, 12, 20, 65, 42, 62, 76, 48, 14, 61, 49, 63,
       76, 56, 42, 37,  4,  3, 46, 74, 78, 83, 61, 42, 16, 12,  5, 26, 10,
        4, 46, 22, 35, 61, 73, 15, 15, 21, 28, 41, 33, 41,  3, 24],
      dtype=int64)

In [ ]:
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())

Input:
 b've all. We will never stop fighting for the sacred values that bind us together as one America we wi'

Next Char Predictions:
 b'Oh4uFpaS0s8BLz?y[8$,PoAW]299qjUDSA\xc2\xb1pOG1\xc2\xa91\xc2\xb1\xc2\xa9j3U (?%\n\nmPrr\xc2\xa9.7lQiwW1hXjwcQL"!Uuy\xc3\x83hQ3.$A,"U9Jht228CPHP!;'


In [ ]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [ ]:
example_batch_mean_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", example_batch_mean_loss)

Prediction shape:  (64, 100, 84)  # (batch_size, sequence_length, vocab_size)
Mean loss:         tf.Tensor(4.431403, shape=(), dtype=float32)


In [ ]:
tf.exp(example_batch_mean_loss).numpy()

84.04927

In [ ]:
model.compile(optimizer='adam', loss=loss)

In [ ]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [ ]:
EPOCHS = 30


In [ ]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])


Epoch 1/30
27/27 [==============================] - 55s 2s/step - loss: 3.7729
Epoch 2/30
27/27 [==============================] - 55s 2s/step - loss: 2.9266
Epoch 3/30
27/27 [==============================] - 54s 2s/step - loss: 2.5507
Epoch 4/30
27/27 [==============================] - 54s 2s/step - loss: 2.3711
Epoch 5/30
27/27 [==============================] - 54s 2s/step - loss: 2.2652
Epoch 6/30
27/27 [==============================] - 55s 2s/step - loss: 2.1743
Epoch 7/30
27/27 [==============================] - 55s 2s/step - loss: 2.0799
Epoch 8/30
27/27 [==============================] - 55s 2s/step - loss: 1.9831
Epoch 9/30
27/27 [==============================] - 55s 2s/step - loss: 1.8931
Epoch 10/30
27/27 [==============================] - 55s 2s/step - loss: 1.8092
Epoch 11/30
27/27 [==============================] - 55s 2s/step - loss: 1.7318
Epoch 12/30
27/27 [==============================] - 55s 2s/step - loss: 1.6588
Epoch 13/30
27/27 [==============================

In [ ]:
model.load_weights('trumpspeeches.h5')

In [ ]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

In [ ]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

In [ ]:
start = time.time()
states = None
next_char = tf.constant(['GREAT:'])
result = [next_char]

for n in range(10000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

NameError: name 'time' is not defined

In [ ]:
def generate_result_string(one_step_model):
  start = time.time()
  states = None
  next_char = tf.constant(['America:'])
  result = [next_char]

  # Initialize a variable to keep track of the result string length
  result_length = 0

  # Set the maximum result string length to 100
  max_result_length = 8400

  while result_length < max_result_length:
    next_char, states = one_step_model.generate_one_step(next_char, states=states)
    result.append(next_char)

    # Update the result string length
    result_length = result_length + 1

  result = tf.strings.join(result)
  end = time.time()
  print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
  print('\nRun time:', end - start)

# Call the function 100 times
for i in range(100):
  generate_result_string(one_step_model)


America: Preace itsod the bokner, thate a veal you. Great your haxpents tigh frow"" [Applause]
Wishing acoucice. Son mase oCforitner if all the ristorien. We willore going to wist to krepitre to keycind in-Butized, hid, it's theeve.
Wh thanke pingro.
They want to take it's tho deally hampentise. Sen'th they whing tlake what he fard thes were as on FOrunemy wat num incummuntion. Bitay deme pelacem broe, commin, and I didn't get ter op ake the lats ince xadening, sboungry ow wirl of there. They dent gongry bigrivent buss read hacilagol you, in juging cas tho Canw.
Betust. We wayt pearcy we're blate tears. They's a mistorigh.
but we're going. But deen toun nigenerencles Apeatcering it's thrig. Ke, wis sanday, he'r you know we Killote, poosh Vy git. I denthous, not mame fichous now now abound ahe way Mxpous to is a rise] Blewe benary tough. I seipt timse] Me're job. No. I cane ander that wo have wiol he'r ghes anit our cexelices an they." pood agratien thet's a latt, betouge thad, all olle

In [ ]:
for n in range(100):
    generate_text()

AttributeError: 'tensorflow.python.framework.ops.EagerTensor' object has no attribute 'append'